## Converting InChi Keys of the compounds on the test dataset into IUPAC names (or to plain English synonyms if available)

### Imports

In [1]:
import pandas as pd
import requests
import time

In [2]:
df = pd.read_csv('../raw_data/tox21_compoundData.csv')

In [3]:
df.head()

,ID,inchikey,sdftitle,order,set,CVfold,NR.AhR,NR.AR,NR.AR.LBD,NR.Aromatase,NR.ER,NR.ER.LBD,NR.PPAR.gamma,SR.ARE,SR.ATAD5,SR.HSE,SR.MMP,SR.p53
0,NCGC00178831-03,PEJLNXHANOHNSU-UHFFFAOYSA-N,NCGC00178831-03,0,training,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN
1,NCGC00166114-03,SEACYXSIPDVVMV-UHFFFAOYSA-L,NCGC00166114-03,1,training,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN
2,NCGC00263563-01,WQGJEAMPBSZCIF-VEKNOCPUSA-N,NCGC00263563-01,2,training,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN
3,NCGC00013058-02,CNYGFPPAGUCRIC-UHFFFAOYSA-L,NCGC00013058-02,3,training,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN
4,NCGC00167516-01,LYCYLGFSIXIXAB-NUZRHMIVSA-N,NCGC00167516-01,4,training,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
df_ids = df[['ID', 'inchikey', 'set']]

In [16]:
df_ids_test = df_ids[df_ids['set'] == 'test']
df_ids_test.reset_index()

,index,ID,inchikey,set
0,12060,NCGC00261900-01,ABCSSKWSUJMJCP-WQDFMEOSSA-N,test
1,12061,NCGC00260869-01,DMRMZQATXPQOTP-XIIVPSJUSA-M,test
2,12062,NCGC00261776-01,ACVGWSKVRYFWRP-UHFFFAOYSA-N,test
3,12063,NCGC00261380-01,LCHACRBDLUKTTM-UHFFFAOYSA-N,test
4,12064,NCGC00261842-01,MQUQNUAYKLCRME-INIZCTEOSA-N,test
...,...,...,...,...
642,12702,NCGC00357168-01,SHLSSLVZXJBVHE-UHFFFAOYSA-N,test
643,12703,NCGC00357283-01,HUHGPYXAVBJSJV-UHFFFAOYSA-N,test
644,12704,NCGC00357210-01,XOHZHMUQBFJTNH-UHFFFAOYSA-N,test
645,12705,NCGC00357118-01,MNHVNIJQQRJYDH-UHFFFAOYSA-N,test
